In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import random
import cv2
import numpy as np
!pip install pyefd
import pyefd
from google.colab.patches import cv2_imshow
!pip install cairocffi
import cairocffi as cairo
import struct
from struct import unpack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.0 MB/s 
  Created wheel for cairocffi: filename=cairocffi-1.4.0-py3-none-any.whl size=88775 sha256=71d6291b5ba3427f20af9359885fc7aea541e32dfd11a0f2cf6537e25719d972
  Stored in directory: /root/.cache/pip/wheels/7a/2b/da/aec872f95d2c24105496ef149a9a576f52daf686f8f2127541
Successfully built cairocffi


In [10]:
# Env vars
torch.use_deterministic_algorithms(False)

# Const vars
LOAD_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN-Fourier hybrid 28.pt'
RAND_SEED = 0
DEVICE = "cuda"

FOURIER_ORDER = 1
IMG_SIDE = 28
IMG_CENTER = np.asarray(((IMG_SIDE - 1) / 2, (IMG_SIDE - 1) / 2))
NUM_CLASSES = 25
EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 512
LOSS_FN = nn.CrossEntropyLoss()

In [11]:
# convert raw vector image to single raster image
def vector_to_raster(vector_image, side=IMG_SIDE, line_diameter=16, padding=80, bg_color=(0,0,0), fg_color=(1,1,1)):
  """
  padding and line_diameter are relative to the original 256x256 image.
  """
  
  original_side = 256.
  
  surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, side, side)
  ctx = cairo.Context(surface)
  ctx.set_antialias(cairo.ANTIALIAS_BEST)
  ctx.set_line_cap(cairo.LINE_CAP_ROUND)
  ctx.set_line_join(cairo.LINE_JOIN_ROUND)
  ctx.set_line_width(line_diameter)

  # scale to match the new size
  # add padding at the edges for the line_diameter
  # and add additional padding to account for antialiasing
  total_padding = padding * 2. + line_diameter
  new_scale = float(side) / float(original_side + total_padding)
  ctx.scale(new_scale, new_scale)
  ctx.translate(total_padding / 2., total_padding / 2.)
      
  bbox = np.hstack(vector_image).max(axis=1)
  offset = ((original_side, original_side) - bbox) / 2.
  offset = offset.reshape(-1,1)
  centered = [stroke + offset for stroke in vector_image]

  # clear background
  ctx.set_source_rgb(*bg_color)
  ctx.paint()

  # draw strokes, this is the most cpu-intensive part
  ctx.set_source_rgb(*fg_color)     
  for xv, yv in centered:   
    ctx.move_to(xv[0], yv[0])
    for x, y in zip(xv, yv):
        ctx.line_to(x, y)
    ctx.stroke()

  data = surface.get_data()
  raster = np.copy(np.asarray(data)[::4]).reshape(side, side)
  return raster

# Define transformation(s) to be applied to dataset-
transforms_norm = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0]
          T.Normalize(mean=(0.138), std=(0.296)) # Quickdraw mean and stdev (35.213, 75.588), divided by 255
      ]
  )

transforms_tensor = T.Compose(
      [
          T.ToTensor(), # scales integer inputs in the range [0, 255] into the range [0.0, 1.0] 
      ]
  )

# transform functions - take sketch image, return torch tensor of descriptors
def transform(vector_img, is_test):
  raster = vector_to_raster(vector_img)

  # add rotations and translations at test time
  if is_test: 
    raster = transforms_tensor(raster.astype(np.float32))

    angle = random.random()*60 - 30
    deltaX = random.randint(-3, 3)
    deltaY = random.randint(-3, 3)

    raster = T.functional.affine(raster, angle, [deltaX, deltaY], 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
    raster = np.squeeze(raster.numpy()).astype(np.uint8)
  
  # fourier transform of outer contour to get rotation angle
  raster_binary = cv2.threshold(raster, 100, 255, cv2.THRESH_BINARY)[1]
  contours, hierarchy = cv2.findContours(raster_binary, 
                                         cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  
  largest_size = 0
  largest_index = 0
  for k, contour in enumerate(contours):
      if len(contour) > largest_size:
        largest_size = len(contour)
        largest_index = k

  contour = np.squeeze(contours[largest_index])
  sketch_center = pyefd.calculate_dc_coefficients(contour)
  coeffs, transform = pyefd.elliptic_fourier_descriptors(contour, order=FOURIER_ORDER, normalize=True, return_transformation=True)
  contour_angle = np.degrees(transform[1])
  img_offset = (IMG_CENTER - sketch_center).round()

  # de-translate and de-rotate
  raster = transforms_norm(raster)
  raster = T.functional.affine(raster, 0, (img_offset[0], img_offset[1]), 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
  raster = T.functional.affine(raster, -1 * contour_angle, [0, 0], 1, 0,
                                 interpolation=T.InterpolationMode.BILINEAR)
  return raster

# helper method to find class based on imgset index
def find_class(idx, num_list):
  class_id = 0
  sum = num_list[class_id]
  while idx >= sum:
    class_id += 1
    sum += num_list[class_id]
  return class_id

# deterministic worker re-seeding
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

# custom dataset for quickdraw
class QuickdrawDataset(Dataset):
  def __init__(self, imgs, nums, is_test):
    self.imgs = imgs
    self.nums = nums
    self.len = sum(nums)
    self.is_test = is_test

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    img = self.imgs[idx]
    x = transform(img, self.is_test)
    y = find_class(idx, self.nums)
    return x, y


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3)
        self.conv2 = nn.Conv2d(64, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.maxpool = nn.MaxPool2d(2) 
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(64 * 4 * 4, 384)
        self.head = nn.Linear(384, NUM_CLASSES)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        return self.head(x)

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding='same')
#         self.conv2 = nn.Conv2d(32, 32, 3, padding='same')
#         self.conv3 = nn.Conv2d(32, 64, 3, padding='same')
#         self.conv4 = nn.Conv2d(64, 64, 3, padding='same')
#         self.conv5 = nn.Conv2d(64, 128, 3, padding='same')
#         self.conv6 = nn.Conv2d(128, 128, 3, padding='same')
#         self.maxpool = nn.MaxPool2d(2) 
#         self.relu = nn.ReLU()
#         self.fc1 = nn.Linear(128 * 4 * 4, 512)
#         self.head = nn.Linear(512, NUM_CLASSES)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = self.conv2(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = self.conv3(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = self.conv4(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = self.conv5(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = self.conv6(x)
#         x = self.maxpool(x)
#         x = self.relu(x)
#         x = torch.flatten(x, 1)
#         x = self.fc1(x)
#         x = self.relu(x)
#         return self.head(x)


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # put the model in train mode
    total_loss = 0
    total_correct = 0
    # for each batch in the training set compute loss and update model parameters
    for batch, (x, y) in enumerate(dataloader):
      x, y = x.to(DEVICE), y.to(DEVICE)
      # Compute prediction and loss
      out = model(x)
      loss = loss_fn(out, y)

      # Backpropagation to update model parameters
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print current training metrics for user
      y, out, loss = y.to("cpu"), out.to("cpu"), loss.to("cpu")
      loss_val = loss.item()
      if batch % 10 == 0:
          current = (batch + 1) * BATCH_SIZE
          print(f"loss: {loss_val:>7f}  [{current:>5d}/{size:>5d}]")

      pred = out.argmax(dim=1, keepdim=True)
      correct = pred.eq(y.view_as(pred)).sum().item()
      total_correct += correct
      total_loss += loss_val
      # print(f"train loss: {loss_val:>7f}   train accuracy: {correct / BATCH_SIZE:>7f}   [batch: {batch + 1:>3d}/{(size // BATCH_SIZE) + 1:>3d}]")      
    print(f"\nepoch avg train loss: {total_loss / ((size // BATCH_SIZE) + 1):>7f}   epoch avg train accuracy: {total_correct / size:>7f}")
      
def eval_loop(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      total_correct += pred.eq(y.view_as(pred)).sum().item()

    accuracy = total_correct / size
    return accuracy


In [5]:
# define methods for unpacking Quickdraw .bin files
def unpack_drawing(file_handle):
  file_handle.read(15)
  n_strokes, = unpack('H', file_handle.read(2))
  image = []
  for i in range(n_strokes):
      n_points, = unpack('H', file_handle.read(2))
      fmt = str(n_points) + 'B'
      x = unpack(fmt, file_handle.read(n_points))
      y = unpack(fmt, file_handle.read(n_points))
      image.append((x, y))

  return image


def unpack_drawings(filename):
  imageset = []
  with open(filename, 'rb') as f:
      while True:
          try:
              imageset.append(unpack_drawing(f))
          except struct.error:
              break
  return imageset

train_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Train/'
test_dir = '/content/drive/My Drive/Fourier/Quickdraw Dataset Small/Test/'
train_imgs = []
test_imgs = []
train_nums = []
test_nums = []
list_of_classes = ["arm", "brain", "circle", "ear", "elbow", "eye", 
                   "face", "finger", "foot", "hand", "hexagon", "knee", 
                   "leg", "line", "mouth", "nose", "octagon", 
                   'skull', 'square', 'squiggle', 'star', 'toe', 'tooth', 
                   'triangle', 'zigzag']
# list_of_classes = ["aircraft carrier", "airplane", "alarm clock", "ambulance", 
#                    "angel", "ant", "anvil", "apple", "arm", "asparagus", "axe", 
#                    "backpack", "banana", "bandage", "barn", "baseball bat", 
#                    "baseball", "basket", "basketball", "bathtub", "beach", "bear", 
#                    "beard", "bed", "bee"]

In [6]:
# load dataset
for item in list_of_classes:
  train_folder = train_dir + item + '.bin'
  test_folder = test_dir + item + '.bin'
  train_drawings = unpack_drawings(train_folder)
  train_imgs += train_drawings
  train_nums.append(len(train_drawings))
  test_drawings = unpack_drawings(test_folder)
  test_imgs += test_drawings
  test_nums.append(len(test_drawings))

In [ ]:
DEVICE = "cpu"
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)
g = torch.Generator()
g.manual_seed(RAND_SEED)

def get_confusion_mat(dataloader, model):
  model.eval()
  size = len(dataloader.dataset)
  conf_mat = np.zeros((NUM_CLASSES, NUM_CLASSES))
  with torch.no_grad():
    total_correct = 0
    for x, y in dataloader:
      x, y = x.to(DEVICE), y.to(DEVICE)
      out = model(x)
      y, out = y.to("cpu"), out.to("cpu")
      pred = out.argmax(dim=1, keepdim=True)
      y = y.view_as(pred)
      total_correct += pred.eq(y).sum().item()
      y, pred = y.numpy(), pred.numpy()
      indices = np.concatenate((y, pred), axis=1)
      for i in range(indices.shape[0]):
        conf_mat[indices[i, 0], indices[i, 1]] += 1

    accuracy = total_correct / size
    print(f"test accuracy: {accuracy:>7f}")
    return conf_mat

eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

model = CNN()
checkpoint = torch.load(LOAD_PATH, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)

random.seed(RAND_SEED)
conf_mat = get_confusion_mat(dataloader=eval_loader,model=model)
conf_mat_norm = conf_mat * (255 / 100)
cv2_imshow(conf_mat_norm)

diag = np.diagonal(conf_mat)
sums = np.asarray(test_nums)
accuracies = diag / sums
acc_sorted_idxs = np.argsort(accuracies)
for i in range(NUM_CLASSES):
  print(list_of_classes[acc_sorted_idxs[i]] + " accuracy " + str(accuracies[acc_sorted_idxs[i]]))

print("\n")

diag_mat = np.multiply(np.identity(NUM_CLASSES), conf_mat)
conf_mat_nodiag = conf_mat - diag_mat
conf_mat_sorted = np.argsort(conf_mat_nodiag, axis=None)
for i in range(1, 6):
  print("class " + list_of_classes[conf_mat_sorted[-i] // NUM_CLASSES] + " misclassified as " + list_of_classes[conf_mat_sorted[-i] % NUM_CLASSES])

test accuracy: 0.705082


octagon accuracy 0.41
knee accuracy 0.48
elbow accuracy 0.49
foot accuracy 0.5
toe accuracy 0.5454545454545454
leg accuracy 0.59
arm accuracy 0.61
squiggle accuracy 0.63
nose accuracy 0.64
finger accuracy 0.67
brain accuracy 0.7
tooth accuracy 0.71
ear accuracy 0.73
face accuracy 0.73
mouth accuracy 0.74
hand accuracy 0.75
hexagon accuracy 0.77
zigzag accuracy 0.79
skull accuracy 0.81
eye accuracy 0.81
triangle accuracy 0.87
circle accuracy 0.9
square accuracy 0.9
star accuracy 0.91
line accuracy 0.94


class octagon misclassified as hexagon
class elbow misclassified as knee
class hexagon misclassified as octagon
class leg misclassified as foot
class knee misclassified as leg


In [12]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN-Fourier hybrid 28.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.218214  [  512/25000]
loss: 2.546573  [ 5632/25000]
loss: 2.239836  [10752/25000]
loss: 2.246207  [15872/25000]
loss: 1.895253  [20992/25000]

epoch avg train loss: 2.329290   epoch avg train accuracy: 0.306920

-------------------------------

Epoch 2

loss: 1.870393  [  512/25000]
loss: 1.814686  [ 5632/25000]
loss: 1.774582  [10752/25000]
loss: 1.619098  [15872/25000]
loss: 1.667599  [20992/25000]

epoch avg train loss: 1.670068   epoch avg train accuracy: 0.512240

-------------------------------

Epoch 3

loss: 1.390854  [  512/25000]
loss: 1.460320  [ 5632/25000]
loss: 1.398507  [10752/25000]
loss: 1.287602  [15872/25000]
loss: 1.383924  [20992/25000]

epoch avg train loss: 1.350327   epoch avg train accuracy: 0.598800

-------------------------------

Epoch 4

loss: 1.212597  [  512/25000]
loss: 1.170589  [ 5632/25000]
loss: 1.323010  [10752/25000]
loss: 1.285719  [15872/25000]
loss: 1.330026  [20992/25000]

epoch avg train loss: 1.205600   epoch avg train accur

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25-2 CNN-Fourier hybrid 256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.221137  [  128/24999]
loss: 2.436512  [ 6528/24999]
loss: 2.238842  [12928/24999]
loss: 1.698128  [19328/24999]

epoch avg train loss: 2.221313   epoch avg train accuracy: 0.335973

-------------------------------

Epoch 2

loss: 1.781789  [  128/24999]
loss: 1.428232  [ 6528/24999]
loss: 1.408190  [12928/24999]
loss: 1.268780  [19328/24999]

epoch avg train loss: 1.322341   epoch avg train accuracy: 0.609424

-------------------------------

Epoch 3

loss: 0.820000  [  128/24999]
loss: 1.172077  [ 6528/24999]
loss: 1.076967  [12928/24999]
loss: 0.891448  [19328/24999]

epoch avg train loss: 1.016731   epoch avg train accuracy: 0.697268

-------------------------------

Epoch 4

loss: 0.921771  [  128/24999]
loss: 0.946426  [ 6528/24999]
loss: 0.905878  [12928/24999]
loss: 0.920459  [19328/24999]

epoch avg train loss: 0.828274   epoch avg train accuracy: 0.755110

-------------------------------

Epoch 5

loss: 0.688732  [  128/24999]
loss: 0.529428  [ 6528/24999]
los

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/Subset25 CNN-Fourier hybrid 256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.215290  [  128/25000]
loss: 2.357484  [ 6528/25000]
loss: 1.836048  [12928/25000]
loss: 1.262836  [19328/25000]

epoch avg train loss: 1.853188   epoch avg train accuracy: 0.443560

-------------------------------

Epoch 2

loss: 1.502856  [  128/25000]
loss: 1.198589  [ 6528/25000]
loss: 1.030130  [12928/25000]
loss: 1.029954  [19328/25000]

epoch avg train loss: 1.132605   epoch avg train accuracy: 0.658080

-------------------------------

Epoch 3

loss: 0.989774  [  128/25000]
loss: 1.043535  [ 6528/25000]
loss: 0.849469  [12928/25000]
loss: 0.832838  [19328/25000]

epoch avg train loss: 0.929748   epoch avg train accuracy: 0.716480

-------------------------------

Epoch 4

loss: 0.723786  [  128/25000]
loss: 0.751628  [ 6528/25000]
loss: 0.817513  [12928/25000]
loss: 0.852584  [19328/25000]

epoch avg train loss: 0.787410   epoch avg train accuracy: 0.757880

-------------------------------

Epoch 5

loss: 0.680723  [  128/25000]
loss: 0.626822  [ 6528/25000]
los

In [ ]:
# seed RNGs
torch.manual_seed(RAND_SEED)
random.seed(RAND_SEED)

# create datasets
train_data = QuickdrawDataset(train_imgs, train_nums, is_test=False)
# eval_data = QuickdrawDataset(test_imgs, test_nums, is_test=False)
test_data = QuickdrawDataset(test_imgs, test_nums, is_test=True)

# create dataloaders
g = torch.Generator()
g.manual_seed(RAND_SEED)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
# eval_loader = DataLoader(eval_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=seed_worker, generator=g)

# init model and optimizer
model = CNN()
# checkpoint = torch.load(SAVE_PATH, map_location=torch.device(DEVICE))
# model.load_state_dict(checkpoint['model_state_dict'])
model.to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# optim.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
epoch = 0

# train for EPOCHS number of epochs then evaluate on test data with affine transformations
SAVE_PATH = '/content/drive/My Drive/Fourier/Saved Models/CNN subset256.pt'
for i in range(epoch, EPOCHS):
    print("Epoch " + str(i + 1) + "\n")
    train_loop(dataloader=train_loader,model=model,loss_fn=LOSS_FN,optimizer=optim)
    # eval_loop(dataloader=eval_loader,model=model)
    torch.save({
                'epoch': i + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optim.state_dict()
                }, SAVE_PATH)
    print("\n-------------------------------\n")
random.seed(RAND_SEED)
accuracies = []
for i in range(30):
  accuracies.append(eval_loop(dataloader=test_loader,model=model))
accuracies = np.asarray(accuracies)
mean = np.mean(accuracies)
std = np.std(accuracies)
print(mean)
print(std)

Epoch 1

loss: 3.333240  [  128/28000]
loss: 2.536796  [ 6528/28000]
loss: 2.020702  [12928/28000]
loss: 1.729918  [19328/28000]
loss: 1.318714  [25728/28000]

epoch avg train loss: 2.003457   epoch avg train accuracy: 0.410679

-------------------------------

Epoch 2

loss: 1.455695  [  128/28000]
loss: 1.322622  [ 6528/28000]
loss: 1.207758  [12928/28000]
loss: 1.417399  [19328/28000]
loss: 1.218234  [25728/28000]

epoch avg train loss: 1.253825   epoch avg train accuracy: 0.630429

-------------------------------

Epoch 3

loss: 0.910056  [  128/28000]
loss: 1.015385  [ 6528/28000]
loss: 1.085217  [12928/28000]
loss: 1.322178  [19328/28000]
loss: 0.855367  [25728/28000]

epoch avg train loss: 1.039027   epoch avg train accuracy: 0.689143

-------------------------------

Epoch 4

loss: 0.885309  [  128/28000]
loss: 0.778443  [ 6528/28000]
loss: 0.784753  [12928/28000]
loss: 0.844654  [19328/28000]
loss: 0.852926  [25728/28000]

epoch avg train loss: 0.880865   epoch avg train accur